In [1]:
print("hi")

hi


In [8]:
import requests
import pandas as pd
from datetime import datetime
import json

def get_all_records_for_date(target_date, max_records=5000):
    """Get all records for a specific date with all available fields"""
    url = "https://boamp-datadila.opendatasoft.com/api/explore/v2.1/catalog/datasets/boamp/records"
    all_records = []
    offset = 0
    limit = 100

    while len(all_records) < max_records:
        params = {
            'order_by': 'dateparution DESC',
            'limit': limit,
            'offset': offset
        }

        print(f"Requesting offset {offset}...")
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            break

        data = response.json()
        records = data.get('results', [])

        if not records:
            break  # No more records

        # Filter records for our target date
        target_records = [record for record in records if record.get('dateparution') == target_date]

        # If we found target records, add them
        if target_records:
            all_records.extend(target_records)
            print(f"Retrieved {len(target_records)} records for {target_date}... Total so far: {len(all_records)}")

        # Check if we've moved past our target date (since we're sorting DESC)
        if records and records[-1].get('dateparution', '') < target_date:
            print(f"Reached dates earlier than {target_date}. Stopping.")
            break

        offset += limit

        if offset > 10000:
            print("Safety limit reached. Stopping.")
            break

    return all_records

In [ ]:
if __name__ == "__main__":
    target_date = '2025-10-31'

    print(f"Searching for BOAMP records with publication date: {target_date}")
    print("=" * 60)

    # Get all records
    all_records = get_all_records_for_date(target_date)

    print(f"\n{'='*60}")
    print(f"EXTRACTION COMPLETE")
    print(f"Found {len(all_records)} records for date {target_date}")

Searching for BOAMP records with publication date: 2025-10-31
Requesting offset 0...
Retrieved 100 records for 2025-10-31... Total so far: 100
Requesting offset 100...
Retrieved 100 records for 2025-10-31... Total so far: 200
Requesting offset 200...
Retrieved 100 records for 2025-10-31... Total so far: 300
Requesting offset 300...
Retrieved 100 records for 2025-10-31... Total so far: 400
Requesting offset 400...
Retrieved 59 records for 2025-10-31... Total so far: 459
Reached dates earlier than 2025-10-31. Stopping.

EXTRACTION COMPLETE
Found 459 records for date 2025-10-31
Searching for BOAMP records with publication date: 2025-10-30
Requesting offset 0...
Requesting offset 100...
Requesting offset 200...
Requesting offset 300...
Requesting offset 400...
Retrieved 41 records for 2025-10-30... Total so far: 41
Requesting offset 500...
Retrieved 100 records for 2025-10-30... Total so far: 141
Requesting offset 600...
Retrieved 100 records for 2025-10-30... Total so far: 241
Requesting 

In [24]:
def create_excel_simple(records, target_date):
    """Simple and robust Excel creation"""

    # Clean the records
    cleaned_records = []
    for record in records:
        cleaned_record = {}
        for key, value in record.items():
            # Handle different data types
            if isinstance(value, (list, dict)):
                cleaned_record[key] = json.dumps(value, ensure_ascii=False)
            elif value is None:
                cleaned_record[key] = ''
            else:
                cleaned_record[key] = value
        cleaned_records.append(cleaned_record)

    # Create DataFrame
    df = pd.DataFrame(cleaned_records)

    # Create filename
    filename = f"BOAMP_{target_date}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"

    # Export to Excel - SIMPLE VERSION, just the data
    df.to_excel(filename, index=False, engine='openpyxl')

    return filename, df

In [25]:
if all_records:
    # Create Excel file
    excel_filename, df = create_excel_simple(all_records, target_date)

    print(f"\n✅ Excel file created: {excel_filename}")
    print(f"📊 Total records exported: {len(all_records)}")
    print(f"📋 Total columns: {len(df.columns)}")

    # Show column names
    print(f"\n📋 Columns in Excel file:")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")

    # Display sample data
    print(f"\n📄 SAMPLE DATA (first 3 records):")
    print(f"{'='*60}")
    for i, record in enumerate(all_records[:3], 1):
        print(f"Record {i}:")
        print(f"  Title: {record.get('objet', 'N/A')[:80]}...")
        print(f"  Buyer: {record.get('nomacheteur', 'N/A')}")
        print(f"  Procedure: {record.get('procedure_libelle', 'N/A')}")
        print(f"  Date: {record.get('dateparution', 'N/A')}")
        print("-" * 60)
else:
    print("❌ No records found for the specified date.")


✅ Excel file created: BOAMP_2025-10-29_20251101_003445.xlsx
📊 Total records exported: 463
📋 Total columns: 41

📋 Columns in Excel file:
   1. idweb
   2. id
   3. contractfolderid
   4. objet
   5. filename
   6. famille
   7. code_departement
   8. code_departement_prestation
   9. famille_libelle
  10. dateparution
  11. datefindiffusion
  12. datelimitereponse
  13. nomacheteur
  14. titulaire
  15. perimetre
  16. type_procedure
  17. soustype_procedure
  18. procedure_libelle
  19. procedure_categorise
  20. nature
  21. sousnature
  22. nature_libelle
  23. sousnature_libelle
  24. nature_categorise
  25. nature_categorise_libelle
  26. criteres
  27. marche_public_simplifie
  28. marche_public_simplifie_label
  29. etat
  30. descripteur_code
  31. dc
  32. descripteur_libelle
  33. type_marche
  34. type_marche_facette
  35. type_avis
  36. annonce_lie
  37. annonces_anterieures
  38. source_schema
  39. gestion
  40. donnees
  41. url_avis

📄 SAMPLE DATA (first 3 records):
Re

In [15]:
import pandas as pd

# --- 1. Define your file path and keyword ---
file_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_20251031_235226.xlsx"
keyword = "serrurerie"
output_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie.xlsx"

# --- 2. Load the Excel file ---
df = pd.read_excel(file_path)

# --- 3. Convert all text to lowercase for case-insensitive search ---
df_str = df.astype(str).apply(lambda x: x.str.lower())

# --- 4. Filter rows that contain the keyword in any column ---
mask = df_str.apply(lambda x: x.str.contains(keyword, na=False))
filtered_df = df[mask.any(axis=1)]

# --- 5. Save the filtered data to a new Excel file ---
filtered_df.to_excel(output_path, index=False)

print(f"✅ Found {len(filtered_df)} matching rows. Saved to:\n{output_path}")


✅ Found 15 matching rows. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie.xlsx


In [18]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# --- 1. Paths & Keyword ---
file_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie.xlsx"
output_path = r"D:\sorabo\sorabo\BOAMP.xlsx"
keyword = "serrurerie"

# --- 2. Load Excel file into pandas ---
df = pd.read_excel(file_path)

# --- 3. Save temporarily (openpyxl works on real Excel files) ---
df.to_excel(output_path, index=False)

# --- 4. Load workbook with openpyxl ---
wb = load_workbook(output_path)
ws = wb.active

# --- 5. Define the green fill style ---
green_fill = PatternFill(start_color="90EE90", end_color="90EE90", fill_type="solid")  # Light green

# --- 6. Loop through each cell and check for keyword ---
for row in ws.iter_rows(min_row=2):  # skip header row
    for cell in row:
        if cell.value and keyword.lower() in str(cell.value).lower():
            cell.fill = green_fill

# --- 7. Save workbook ---
wb.save(output_path)

print(f"✅ Highlighted all cells containing '{keyword}' in green.\nSaved to:\n{output_path}")


✅ Highlighted all cells containing 'serrurerie' in green.
Saved to:
D:\sorabo\sorabo\BOAMP.xlsx


In [19]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# --- File paths ---
input_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie.xlsx"
output_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie_highlighted.xlsx"

# --- Keyword to highlight ---
keyword = "serrurerie"

# --- Load Excel file with openpyxl directly ---
wb = load_workbook(input_path)
ws = wb.active

# --- Define the green fill ---
green_fill = PatternFill(start_color="90EE90", end_color="90EE90", fill_type="solid")

# --- Loop through cells and apply highlight ---
count = 0
for row in ws.iter_rows(min_row=1):  # Include headers if needed
    for cell in row:
        if cell.value and keyword.lower() in str(cell.value).lower():
            cell.fill = green_fill
            count += 1

# --- Save the new file ---
wb.save(output_path)

print(f"✅ Highlighted {count} cells containing '{keyword}'.")
print(f"💾 Saved to: {output_path}")


✅ Highlighted 17 cells containing 'serrurerie'.
💾 Saved to: D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie_highlighted.xlsx


In [20]:
import pandas as pd
import os

# --- 1. Define paths and keywords ---
file_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_20251031_235226.xlsx"
output_dir = r"D:\sorabo\sorabo"
keywords = [
    "serrurerie",
    "miroiterie",
    "métallerie",
    "menuiserie extérieure",
    "clôtures",
    "44233000",
    "escaliers",
    "44316500",
    "45421132",
    "45421140"
]

# --- 2. Load the Excel file ---
df = pd.read_excel(file_path)
df_str = df.astype(str).apply(lambda x: x.str.lower())  # lowercase for case-insensitive match

# --- 3. Prepare a list to store all matches ---
all_matches = pd.DataFrame()

# --- 4. Loop through each keyword ---
for keyword in keywords:
    mask = df_str.apply(lambda x: x.str.contains(keyword.lower(), na=False))
    filtered_df = df[mask.any(axis=1)]
    
    if not filtered_df.empty:
        output_path = os.path.join(output_dir, f"BOAMP_2025-10-31_{keyword.replace(' ', '_')}.xlsx")
        filtered_df.to_excel(output_path, index=False)
        print(f"✅ Found {len(filtered_df)} rows for '{keyword}'. Saved to:\n{output_path}")
        
        # Add a column showing which keyword matched
        filtered_df = filtered_df.copy()
        filtered_df["keyword"] = keyword
        all_matches = pd.concat([all_matches, filtered_df], ignore_index=True)
    else:
        print(f"⚠️ No matches found for '{keyword}'.")

# --- 5. Save all combined matches ---
if not all_matches.empty:
    combined_output = os.path.join(output_dir, "BOAMP_2025-10-31_ALL_KEYWORDS.xlsx")
    all_matches.to_excel(combined_output, index=False)
    print(f"\n📊 Combined results saved to:\n{combined_output}")
else:
    print("\n❌ No matches found for any keyword.")


✅ Found 15 rows for 'serrurerie'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_serrurerie.xlsx
⚠️ No matches found for 'miroiterie'.
✅ Found 7 rows for 'métallerie'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_métallerie.xlsx
✅ Found 4 rows for 'menuiserie extérieure'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_menuiserie_extérieure.xlsx
✅ Found 3 rows for 'clôtures'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_clôtures.xlsx
⚠️ No matches found for '44233000'.
✅ Found 1 rows for 'escaliers'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_escaliers.xlsx
✅ Found 4 rows for '44316500'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_44316500.xlsx
⚠️ No matches found for '45421132'.
✅ Found 2 rows for '45421140'. Saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_45421140.xlsx

📊 Combined results saved to:
D:\sorabo\sorabo\BOAMP_2025-10-31_ALL_KEYWORDS.xlsx


In [18]:
import pandas as pd
import os

# --- 1. Define your file path ---
file_path = r"D:\sorabo\sorabo\data\31-10_merged.xlsx"
output_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_ALL_KEYWORDS.xlsx"

# --- 2. Define all your keywords and CPV codes ---
keywords = [
    # Secteurs d’activité
    "serrurerie",
    "miroiterie",
    "métallerie",
    "menuiserie extérieure",
    "clôtures",
    "escaliers",

    # CPV simples
    "44233000",
    "44316500",
    "45421132",
    "45421140",

    # CPV détaillés avec description
    "45420000-7", "Travaux de menuiserie et de charpenterie",
    "45421000-4", "Travaux de menuiserie",
    "45421100-5", "Pose de portes et de fenêtres et d'éléments accessoires",
    "45421110-8", "Pose d'encadrements de portes et de fenêtres",
    "45421111-5", "Pose d'encadrements de portes",
    "45421112-2", "Pose d'encadrements de fenêtres",
    "45421120-1", "Pose de seuils",
    "45421130-4", "Poses de portes et de fenêtres",
    "45421131-1", "Pose de portes",
    "45421132-8", "Pose de fenêtres",
    "45421140-7", "Pose de menuiseries métalliques, excepté portes et fenêtres",
    "45421141-4", "Travaux de cloisonnement",
    "45421142-1", "Installation de volets",
    "45421143-8", "Travaux d'installation de stores",
    "45421144-5", "Travaux d'installation de vélums",
    "45421145-2", "Travaux d'installation de volets roulants"
]

# --- 3. Load Excel ---
print("📂 Reading Excel file...")
df = pd.read_excel(file_path)
df_str = df.astype(str).apply(lambda x: x.str.lower())  # for case-insensitive search

# --- 4. Create one big dataframe for all results ---
all_matches = pd.DataFrame()

# --- 5. Loop through each keyword ---
for keyword in keywords:
    mask = df_str.apply(lambda x: x.str.contains(keyword.lower(), na=False))
    filtered_df = df[mask.any(axis=1)]
    
    if not filtered_df.empty:
        print(f"✅ Found {len(filtered_df)} rows for '{keyword}'")
        filtered_df = filtered_df.copy()
        filtered_df["keyword"] = keyword
        all_matches = pd.concat([all_matches, filtered_df], ignore_index=True)
    else:
        print(f"⚠️ No matches found for '{keyword}'.")

# --- 6. Save everything to one file ---
if not all_matches.empty:
    all_matches.to_excel(output_path, index=False)
    print(f"\n📊 All matching data saved successfully to:\n{output_path}")
else:
    print("\n❌ No matches found for any keyword.")


📂 Reading Excel file...
✅ Found 257 rows for 'serrurerie'
✅ Found 11 rows for 'miroiterie'
✅ Found 95 rows for 'métallerie'
✅ Found 30 rows for 'menuiserie extérieure'
✅ Found 52 rows for 'clôtures'
✅ Found 23 rows for 'escaliers'
⚠️ No matches found for '44233000'.
✅ Found 73 rows for '44316500'
⚠️ No matches found for '45421132'.
✅ Found 16 rows for '45421140'
⚠️ No matches found for '45420000-7'.
⚠️ No matches found for 'Travaux de menuiserie et de charpenterie'.
✅ Found 3 rows for '45421000-4'
✅ Found 23 rows for 'Travaux de menuiserie'
⚠️ No matches found for '45421100-5'.
⚠️ No matches found for 'Pose de portes et de fenêtres et d'éléments accessoires'.
⚠️ No matches found for '45421110-8'.
⚠️ No matches found for 'Pose d'encadrements de portes et de fenêtres'.
⚠️ No matches found for '45421111-5'.
⚠️ No matches found for 'Pose d'encadrements de portes'.
⚠️ No matches found for '45421112-2'.
⚠️ No matches found for 'Pose d'encadrements de fenêtres'.
⚠️ No matches found for '45421

In [4]:

import pandas as pd
import os

# --- 1. Define your file path ---
file_path = r"D:\sorabo\sorabo\data\codes.xlsx"
output_path = r"D:\sorabo\sorabo\codesc.xlsx"

# --- 2. Define all your keywords and CPV codes ---
keywords = [
    # Secteurs d’activité
    "serrurerie",
    "miroiterie",
    "métallerie",
    "menuiserie",
    "clôtures",
    "escaliers",

]

# --- 3. Load Excel ---
print("📂 Reading Excel file...")
df = pd.read_excel(file_path)
df_str = df.astype(str).apply(lambda x: x.str.lower())  # for case-insensitive search

# --- 4. Create one big dataframe for all results ---
all_matches = pd.DataFrame()

# --- 5. Loop through each keyword ---
for keyword in keywords:
    mask = df_str.apply(lambda x: x.str.contains(keyword.lower(), na=False))
    filtered_df = df[mask.any(axis=1)]
    
    if not filtered_df.empty:
        print(f"✅ Found {len(filtered_df)} rows for '{keyword}'")
        filtered_df = filtered_df.copy()
        filtered_df["keyword"] = keyword
        all_matches = pd.concat([all_matches, filtered_df], ignore_index=True)
    else:
        print(f"⚠️ No matches found for '{keyword}'.")

# --- 6. Save everything to one file ---
if not all_matches.empty:
    all_matches.to_excel(output_path, index=False)
    print(f"\n📊 All matching data saved successfully to:\n{output_path}")
else:
    print("\n❌ No matches found for any keyword.")


📂 Reading Excel file...
✅ Found 2 rows for 'serrurerie'
⚠️ No matches found for 'miroiterie'.
⚠️ No matches found for 'métallerie'.
✅ Found 5 rows for 'menuiserie'
✅ Found 4 rows for 'clôtures'
✅ Found 10 rows for 'escaliers'

📊 All matching data saved successfully to:
D:\sorabo\sorabo\codesc.xlsx


In [27]:
import requests
import pandas as pd
from datetime import datetime
import json

def get_all_records_for_date(target_date, max_records=5000):
    """Get all records for a specific date with all available fields"""
    url = "https://boamp-datadila.opendatasoft.com/api/explore/v2.1/catalog/datasets/boamp/records"
    all_records = []
    offset = 0
    limit = 100

    while len(all_records) < max_records:
        params = {
            'order_by': 'dateparution DESC',
            'limit': limit,
            'offset': offset
        }

        print(f"Requesting offset {offset}...")
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            break

        data = response.json()
        records = data.get('results', [])

        if not records:
            break  # No more records

        # Filter records for our target date
        target_records = [record for record in records if record.get('dateparution') == target_date]

        # If we found target records, add them
        if target_records:
            all_records.extend(target_records)
            print(f"Retrieved {len(target_records)} records for {target_date}... Total so far: {len(all_records)}")

        # Check if we've moved past our target date (since we're sorting DESC)
        if records and records[-1].get('dateparution', '') < target_date:
            print(f"Reached dates earlier than {target_date}. Stopping.")
            break

        offset += limit

        if offset > 10000:
            print("Safety limit reached. Stopping.")
            break

    return all_records


if __name__ == "__main__":
    # Multiple dates
    target_dates = ['2025-10-31', '2025-10-30', '2025-10-29', '2025-10-28', '2025-10-27']
    all_results = []

    for target_date in target_dates:
        print(f"\n{'='*60}")
        print(f"Searching for BOAMP records with publication date: {target_date}")
        print("=" * 60)

        records = get_all_records_for_date(target_date)
        print(f"Found {len(records)} records for date {target_date}")
        all_results.extend(records)

    # Save all data in one JSON file
    output_file = f"boamp_records_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)

    print(f"\n{'='*60}")
    print(f"✅ EXTRACTION COMPLETE — Total {len(all_results)} records from {len(target_dates)} dates.")
    print(f"💾 Data saved to {output_file}")



Searching for BOAMP records with publication date: 2025-10-31
Requesting offset 0...
Retrieved 100 records for 2025-10-31... Total so far: 100
Requesting offset 100...
Retrieved 100 records for 2025-10-31... Total so far: 200
Requesting offset 200...
Retrieved 100 records for 2025-10-31... Total so far: 300
Requesting offset 300...
Retrieved 100 records for 2025-10-31... Total so far: 400
Requesting offset 400...
Retrieved 59 records for 2025-10-31... Total so far: 459
Reached dates earlier than 2025-10-31. Stopping.
Found 459 records for date 2025-10-31

Searching for BOAMP records with publication date: 2025-10-30
Requesting offset 0...
Requesting offset 100...
Requesting offset 200...
Requesting offset 300...
Requesting offset 400...
Retrieved 41 records for 2025-10-30... Total so far: 41
Requesting offset 500...
Retrieved 100 records for 2025-10-30... Total so far: 141
Requesting offset 600...
Retrieved 100 records for 2025-10-30... Total so far: 241
Requesting offset 700...
Retri

In [28]:
import requests
import pandas as pd
from datetime import datetime

def get_all_records_for_date(target_date, max_records=5000):
    """Get all records for a specific date with all available fields"""
    url = "https://boamp-datadila.opendatasoft.com/api/explore/v2.1/catalog/datasets/boamp/records"
    all_records = []
    offset = 0
    limit = 100

    while len(all_records) < max_records:
        params = {
            'order_by': 'dateparution DESC',
            'limit': limit,
            'offset': offset
        }

        print(f"Requesting offset {offset}...")
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            break

        data = response.json()
        records = data.get('results', [])

        if not records:
            break  # No more records

        # Filter records for our target date
        target_records = [record for record in records if record.get('dateparution') == target_date]

        if target_records:
            all_records.extend(target_records)
            print(f"Retrieved {len(target_records)} records for {target_date}... Total so far: {len(all_records)}")

        # Stop if we’ve reached dates earlier than target_date
        if records and records[-1].get('dateparution', '') < target_date:
            print(f"Reached dates earlier than {target_date}. Stopping.")
            break

        offset += limit

        if offset > 10000:
            print("Safety limit reached. Stopping.")
            break

    return all_records


if __name__ == "__main__":
    target_dates = ['2025-10-31', '2025-10-30', '2025-10-29', '2025-10-28', '2025-10-27']
    all_results = []

    for target_date in target_dates:
        print(f"\n{'='*60}")
        print(f"Fetching BOAMP records for: {target_date}")
        records = get_all_records_for_date(target_date)
        print(f"→ Found {len(records)} records for {target_date}")
        all_results.extend(records)

    print(f"\nTotal records retrieved: {len(all_results)}")

    # ✅ Save results to Excel
    if all_results:
        df = pd.DataFrame(all_results)
        output_file = f"boamp_records_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df.to_excel(output_file, index=False, engine='openpyxl')
        print(f"💾 Data saved to Excel file: {output_file}")
    else:
        print("⚠️ No records found for the given dates.")



Fetching BOAMP records for: 2025-10-31
Requesting offset 0...
Retrieved 100 records for 2025-10-31... Total so far: 100
Requesting offset 100...
Retrieved 100 records for 2025-10-31... Total so far: 200
Requesting offset 200...
Retrieved 100 records for 2025-10-31... Total so far: 300
Requesting offset 300...
Retrieved 100 records for 2025-10-31... Total so far: 400
Requesting offset 400...
Retrieved 59 records for 2025-10-31... Total so far: 459
Reached dates earlier than 2025-10-31. Stopping.
→ Found 459 records for 2025-10-31

Fetching BOAMP records for: 2025-10-30
Requesting offset 0...
Requesting offset 100...
Requesting offset 200...
Requesting offset 300...
Requesting offset 400...
Retrieved 41 records for 2025-10-30... Total so far: 41
Requesting offset 500...
Retrieved 100 records for 2025-10-30... Total so far: 141
Requesting offset 600...
Retrieved 100 records for 2025-10-30... Total so far: 241
Requesting offset 700...
Retrieved 100 records for 2025-10-30... Total so far: 

In [2]:
import requests
import pandas as pd
from datetime import datetime

def get_all_records_for_date(target_date, max_records=5000):
    """Get all records for a specific date with all available fields"""
    url = "https://boamp-datadila.opendatasoft.com/api/explore/v2.1/catalog/datasets/boamp/records"
    all_records = []
    offset = 0
    limit = 100

    while len(all_records) < max_records:
        params = {
            'order_by': 'dateparution DESC',
            'limit': limit,
            'offset': offset
        }

        print(f"Requesting offset {offset}...")
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            break

        data = response.json()
        records = data.get('results', [])

        if not records:
            break  # No more records

        # Filter records for our target date
        target_records = [record for record in records if record.get('dateparution') == target_date]

        if target_records:
            all_records.extend(target_records)
            print(f"Retrieved {len(target_records)} records for {target_date}... Total so far: {len(all_records)}")

        # Stop if we’ve reached dates earlier than target_date
        if records and records[-1].get('dateparution', '') < target_date:
            print(f"Reached dates earlier than {target_date}. Stopping.")
            break

        offset += limit

        if offset > 10000:
            print("Safety limit reached. Stopping.")
            break

    return all_records


if __name__ == "__main__":
    target_dates = ['2025-10-19', '2025-10-18', '2025-10-17', '2025-10-16', '2025-10-15', '2025-10-14', '2025-10-13', '2025-10-12', '2025-10-11', '2025-10-10']
    all_results = []

    for target_date in target_dates:
        print(f"\n{'='*60}")
        print(f"Fetching BOAMP records for: {target_date}")
        records = get_all_records_for_date(target_date)
        print(f"→ Found {len(records)} records for {target_date}")
        all_results.extend(records)

    print(f"\nTotal records retrieved: {len(all_results)}")

    # ✅ Save results to Excel
    if all_results:
        df = pd.DataFrame(all_results)
        output_file = f"boamp_records_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df.to_excel(output_file, index=False, engine='openpyxl')
        print(f"💾 Data saved to Excel file: {output_file}")
    else:
        print("⚠️ No records found for the given dates.")



Fetching BOAMP records for: 2025-10-19
Requesting offset 0...


ConnectionError: HTTPSConnectionPool(host='boamp-datadila.opendatasoft.com', port=443): Max retries exceeded with url: /api/explore/v2.1/catalog/datasets/boamp/records?order_by=dateparution+DESC&limit=100&offset=0 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000259E9C9B700>: Failed to resolve 'boamp-datadila.opendatasoft.com' ([Errno 11001] getaddrinfo failed)"))

In [12]:
from datetime import datetime, timedelta

dates_octobre = [
    (datetime(2025, 10, 19) - timedelta(days=i)).strftime("%Y-%m-%d")
    for i in range(10)
]

print(dates_octobre)


['2025-10-19', '2025-10-18', '2025-10-17', '2025-10-16', '2025-10-15', '2025-10-14', '2025-10-13', '2025-10-12', '2025-10-11', '2025-10-10']


In [16]:
import pandas as pd

# Paths of the two Excel files
file1 = r"D:\sorabo\sorabo\data\19-10.xlsx"
file2 = r"D:\sorabo\sorabo\data\31-20.xlsx"

# Read both Excel files
df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

# Combine the two dataframes
merged_df = pd.concat([df1, df2], ignore_index=True)

# Save the merged file (overwrite or create new one)
output_path = r"D:\sorabo\sorabo\data\31-10_merged.xlsx"
merged_df.to_excel(output_path, index=False, engine='openpyxl')

print(f"✅ Files merged successfully and saved as: {output_path}")


✅ Files merged successfully and saved as: D:\sorabo\sorabo\data\31-10_merged.xlsx


In [3]:
import pandas as pd

files = [
    r"D:\sorabo\sorabo\BOAMP_2025-10-31_ALL_KEYWORDS2.xlsx",
    r"D:\sorabo\sorabo\data\octobre.xlsx"
]

for file in files:
    try:
        df = pd.read_excel(file)
        print(f"📘 {file} → {len(df)} rows")
    except FileNotFoundError:
        print(f"❌ File not found: {file}")
    except Exception as e:
        print(f"⚠️ Error reading {file}: {e}")


📘 D:\sorabo\sorabo\BOAMP_2025-10-31_ALL_KEYWORDS2.xlsx → 1119 rows
📘 D:\sorabo\sorabo\data\octobre.xlsx → 10000 rows


In [6]:
import pandas as pd

# Path to your Excel file
file_path = r"D:\sorabo\sorabo\codesc.xlsx"

# Read the Excel file
df = pd.read_excel(file_path)

# Use the correct column names
code_col = "CODE"      # CPV code column
desc_col = "FR"        # French description column

# Create the list
cpv_list = []
for _, row in df.iterrows():
    code = str(row[code_col]).strip()
    desc = str(row[desc_col]).strip()
    cpv_list.append(code)
    cpv_list.append(desc)

# Print result in your desired format
print("\n# CPV détaillés avec description")
print("cpv_list = [")
for i in range(0, len(cpv_list), 2):
    print(f'    "{cpv_list[i]}", "{cpv_list[i+1]}",')
print("]")



# CPV détaillés avec description
cpv_list = [
    "44316500-3", "Serrurerie",
    "98395000-8", "Services de serrurerie",
    "44220000-8", "Menuiserie pour la construction",
    "45420000-7", "Travaux de menuiserie et de charpenterie",
    "45421000-4", "Travaux de menuiserie",
    "45421140-7", "Pose de menuiseries métalliques, excepté portes et fenêtres",
    "45421150-0", "Travaux d'installation de menuiseries non métalliques",
    "34928200-0", "Clôtures",
    "34928310-4", "Clôtures de protection",
    "45340000-2", "Travaux d'installation de clôtures, de garde-corps et de dispositifs de sécurité",
    "45342000-6", "Pose de clôtures",
    "42416000-5", "Ascenseurs, skips, monte-charges, escaliers mécaniques et trottoirs roulants",
    "42416400-9", "Escaliers mécaniques",
    "42419500-1", "Pièces pour ascenseurs, skips ou escaliers mécaniques",
    "42419530-0", "Pièces pour escaliers mécaniques",
    "44233000-2", "Escaliers",
    "44423220-9", "Escaliers pliants",
    "45313

In [ ]:
import pandas as pd
import os

# --- 1. Define your file path ---
file_path = r"D:\sorabo\sorabo\data\31-10_merged.xlsx"
output_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_ALL_KEYWORDS.xlsx"

# --- 2. Define all your keywords and CPV codes ---
keywords = [
    # Secteurs d’activité
    "miroiterie",
    "métallerie",
    "menuiserie extérieure",

    # CPV simples
    
   
    
  

    # CPV détaillés avec description
    "45420000", "Travaux de menuiserie et de charpenterie",
    "45421100", "Pose de portes et de fenêtres et d'éléments accessoires",
    "45421110", "Pose d'encadrements de portes et de fenêtres",
    "45421111", "Pose d'encadrements de portes",
    "45421112", "Pose d'encadrements de fenêtres",
    "45421120", "Pose de seuils",
    "45421130", "Poses de portes et de fenêtres",
    "45421131", "Pose de portes",
    "45421132", "Pose de fenêtres",
    "45421140", "Pose de menuiseries métalliques, excepté portes et fenêtres",
    "45421141", "Travaux de cloisonnement",
    "45421142", "Installation de volets",
    "45421143", "Travaux d'installation de stores",
    "45421144", "Travaux d'installation de vélums",
    "45421145", "Travaux d'installation de volets roulants",
    "44316500", "Serrurerie",
    "98395000", "Services de serrurerie",
    "44220000", "Menuiserie pour la construction",
    "45421000", "Travaux de menuiserie",
    "45421140", "Pose de menuiseries métalliques, excepté portes et fenêtres",
    "45421150", "Travaux d'installation de menuiseries non métalliques",
    "34928200", "Clôtures",
    "34928310", "Clôtures de protection",
    "45340000", "Travaux d'installation de clôtures, de garde-corps et de dispositifs de sécurité",
    "45342000", "Pose de clôtures",
    "42416000", "Ascenseurs, skips, monte-charges, escaliers mécaniques et trottoirs roulants",
    "42416400", "Escaliers mécaniques",
    "42419500", "Pièces pour ascenseurs, skips ou escaliers mécaniques",
    "42419530", "Pièces pour escaliers mécaniques",
    "44233000", "Escaliers",
    "44423220", "Escaliers pliants",
    "45313000", "Travaux d'installation d'ascenseurs et d'escaliers mécaniques",
    "45313200", "Travaux d'installation d'escaliers mécaniques",
    "50740000", "Services de réparation et d'entretien d'escaliers mécaniques",
    "51511000", "Services d'installation de matériel de levage et de manutention, excepté ascenseurs et escaliers mécaniques",]
# --- 3. Loadxcel ---
print("📂 Reang Excel file...")
df = pd.read_excel(file_path)
df_str = df.astype(str).apply(lambda x: x.str.lower())  # for case-insensitive search

# --- 4. Create one big dataframe for all results ---
all_matches = pd.DataFrame()

# --- 5. Loop through each keyword ---
for keyword in keywords:
    mask = df_str.apply(lambda x: x.str.contains(keyword.lower(), na=False))
    filtered_df = df[mask.any(axis=1)]
    
    if not filtered_df.empty:
        print(f"✅ Found {len(filtered_df)} rows for '{keyword}'")
        filtered_df = filtered_df.copy()
        filtered_df["keyword"] = keyword
        all_matches = pd.concat([all_matches, filtered_df], ignore_index=True)
    else:
        print(f"⚠️ No matches found for '{keyword}'.")

# --- 6. Save everything to one file ---
if not all_matches.empty:
    all_matches.to_excel(output_path, index=False)
    print(f"\n📊 All matching data saved successfully to:\n{output_path}")
else:
    print("\n❌ No matches found for any keyword.")


📂 Reading Excel file...
✅ Found 14 rows for 'miroiterie'
✅ Found 110 rows for 'métallerie'
✅ Found 33 rows for 'menuiserie extérieure'
✅ Found 29 rows for '45420000'
⚠️ No matches found for 'Travaux de menuiserie et de charpenterie'.
✅ Found 16 rows for '45421100'
⚠️ No matches found for 'Pose de portes et de fenêtres et d'éléments accessoires'.
✅ Found 1 rows for '45421110'
⚠️ No matches found for 'Pose d'encadrements de portes et de fenêtres'.
✅ Found 1 rows for '45421111'
⚠️ No matches found for 'Pose d'encadrements de portes'.
✅ Found 2 rows for '45421112'
⚠️ No matches found for 'Pose d'encadrements de fenêtres'.
⚠️ No matches found for '45421120'.
⚠️ No matches found for 'Pose de seuils'.
✅ Found 15 rows for '45421130'
⚠️ No matches found for 'Poses de portes et de fenêtres'.
✅ Found 3 rows for '45421131'
✅ Found 1 rows for 'Pose de portes'
✅ Found 1 rows for '45421132'
⚠️ No matches found for 'Pose de fenêtres'.
✅ Found 17 rows for '45421140'
⚠️ No matches found for 'Pose de men

In [11]:
import pandas as pd

# ✅ 1. Path to your Excel file
file_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_ALL_KEYWORDSwithout verefication.xlsx"


# === 2️⃣ Read the Excel file ===
df = pd.read_excel(file_path)

# === 3️⃣ Column name that contains the ID ===
# Change 'CODE' to your actual column name if different
id_column = 'id'

# === 4️⃣ Find duplicated IDs ===
duplicated_ids = df[id_column][df[id_column].duplicated(keep=False)]

# === 5️⃣ Remove all rows with those duplicated IDs ===
df_clean = df[~df[id_column].isin(duplicated_ids)]

# === 6️⃣ Save to a new Excel file ===
output_path = r"D:\sorabo\sorabo\data\31-10_merged_no_duplicates.xlsx"
df_clean.to_excel(output_path, index=False)

print(f"✅ Done! Removed {len(df) - len(df_clean)} duplicated rows.")
print(f"💾 Clean file saved to: {output_path}")


✅ Done! Removed 920 duplicated rows.
💾 Clean file saved to: D:\sorabo\sorabo\data\31-10_merged_no_duplicates.xlsx


In [12]:
import pandas as pd

# === 1️⃣ Path to your Excel file ===
file_path = r"D:\sorabo\sorabo\BOAMP_2025-10-31_ALL_KEYWORDSwithout verefication.xlsx"

# === 2️⃣ Read the Excel file ===
df = pd.read_excel(file_path)

# === 3️⃣ Column name containing the ID ===
# Change this if your ID column has a different name
id_column = 'id'

# === 4️⃣ Remove duplicate rows — keep only the first occurrence per ID ===
df_clean = df.drop_duplicates(subset=[id_column], keep='first')

# === 5️⃣ Save to a new Excel file ===
output_path = r"D:\sorabo\sorabo\data\new.xlsx"
df_clean.to_excel(output_path, index=False)

print(f"✅ Done! Kept {len(df_clean)} unique rows out of {len(df)} total.")
print(f"💾 Clean file saved to: {output_path}")


✅ Done! Kept 628 unique rows out of 1216 total.
💾 Clean file saved to: D:\sorabo\sorabo\data\new.xlsx


In [16]:
import requests
import pandas as pd
from datetime import datetime
import json

def get_all_records_for_date(target_date, max_records=5000):
    """Get all records for a specific date with all available fields"""
    url = "https://boamp-datadila.opendatasoft.com/api/explore/v2.1/catalog/datasets/boamp/records"
    all_records = []
    offset = 0
    limit = 100

    while len(all_records) < max_records:
        params = {
            'order_by': 'dateparution DESC',
            'limit': limit,
            'offset': offset
        }

        print(f"Requesting offset {offset}...")
        response = requests.get(url, params=params)

        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            break

        data = response.json()
        records = data.get('results', [])

        if not records:
            break  # No more records

        # Filter records for our target date
        target_records = [record for record in records if record.get('dateparution') == target_date]

        # If we found target records, add them
        if target_records:
            all_records.extend(target_records)
            print(f"Retrieved {len(target_records)} records for {target_date}... Total so far: {len(all_records)}")

        # Check if we've moved past our target date (since we're sorting DESC)
        if records and records[-1].get('dateparution', '') < target_date:
            print(f"Reached dates earlier than {target_date}. Stopping.")
            break

        offset += limit

        if offset > 10000:
            print("Safety limit reached. Stopping.")
            break

    return all_records

def create_excel_simple(records, target_date):
    """Simple and robust Excel creation"""

    # Clean the records
    cleaned_records = []
    for record in records:
        cleaned_record = {}
        for key, value in record.items():
            # Handle different data types
            if isinstance(value, (list, dict)):
                cleaned_record[key] = json.dumps(value, ensure_ascii=False)
            elif value is None:
                cleaned_record[key] = ''
            else:
                cleaned_record[key] = value
        cleaned_records.append(cleaned_record)

    # Create DataFrame
    df = pd.DataFrame(cleaned_records)

    # Create filename
    filename = f"BOAMP_{target_date}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"

    # Export to Excel - SIMPLE VERSION, just the data
    df.to_excel(filename, index=False, engine='openpyxl')

    return filename, df

# Main execution
if __name__ == "__main__":
    target_date = '2025-10-31'

    print(f"Searching for BOAMP records with publication date: {target_date}")
    print("=" * 60)

    # Get all records
    all_records = get_all_records_for_date(target_date)

    print(f"\n{'='*60}")
    print(f"EXTRACTION COMPLETE")
    print(f"Found {len(all_records)} records for date {target_date}")

    if all_records:
        # Create Excel file
        excel_filename, df = create_excel_simple(all_records, target_date)

        print(f"\n✅ Excel file created: {excel_filename}")
        print(f"📊 Total records exported: {len(all_records)}")
        print(f"📋 Total columns: {len(df.columns)}")

        # Show column names
        print(f"\n📋 Columns in Excel file:")
        for i, col in enumerate(df.columns, 1):
            print(f"  {i:2d}. {col}")

        # Display sample data
        print(f"\n📄 SAMPLE DATA (first 3 records):")
        print(f"{'='*60}")
        for i, record in enumerate(all_records[:3], 1):
            print(f"Record {i}:")
            print(f"  Title: {record.get('objet', 'N/A')[:80]}...")
            print(f"  Buyer: {record.get('nomacheteur', 'N/A')}")
            print(f"  Procedure: {record.get('procedure_libelle', 'N/A')}")
            print(f"  Date: {record.get('dateparution', 'N/A')}")
            print("-" * 60)
    else:
        print("❌ No records found for the specified date.")



Searching for BOAMP records with publication date: 2025-10-31
Requesting offset 0...
Requesting offset 100...
Requesting offset 200...
Retrieved 56 records for 2025-10-31... Total so far: 56
Requesting offset 300...
Retrieved 100 records for 2025-10-31... Total so far: 156
Requesting offset 400...
Retrieved 100 records for 2025-10-31... Total so far: 256
Requesting offset 500...
Retrieved 100 records for 2025-10-31... Total so far: 356
Requesting offset 600...
Retrieved 100 records for 2025-10-31... Total so far: 456
Requesting offset 700...
Retrieved 3 records for 2025-10-31... Total so far: 459
Reached dates earlier than 2025-10-31. Stopping.

EXTRACTION COMPLETE
Found 459 records for date 2025-10-31

✅ Excel file created: BOAMP_2025-10-31_20251102_144920.xlsx
📊 Total records exported: 459
📋 Total columns: 41

📋 Columns in Excel file:
   1. idweb
   2. id
   3. contractfolderid
   4. objet
   5. filename
   6. famille
   7. code_departement
   8. code_departement_prestation
   9. fam

In [17]:
def get_excel_row_counts(file_path):
  
    try:
        df = pd.read_excel(file_path)
        print(f"📘 {file_path} → {len(df)} rows")
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
    except Exception as e:
        print(f"⚠️ Error reading {file_path}: {e}")

get_excel_row_counts(excel_filename)


📘 BOAMP_2025-10-31_20251102_144920.xlsx → 459 rows


In [5]:
get_excel_row_counts(excel_filename)

📘 BOAMP_2025-10-29_20251102_140744.xlsx → 463 rows


In [18]:
def get_keword_excel(file_path, output_path, keywords):
   
    print("📂 Reang Excel file...")
    df = pd.read_excel(file_path)
    df_str = df.astype(str).apply(lambda x: x.str.lower())  # for case-insensitive search

    # --- 4. Create one big dataframe for all results ---
    all_matches = pd.DataFrame()

    # --- 5. Loop through each keyword ---
    for keyword in keywords:
        mask = df_str.apply(lambda x: x.str.contains(keyword.lower(), na=False))
        filtered_df = df[mask.any(axis=1)]
        
        if not filtered_df.empty:
            print(f"✅ Found {len(filtered_df)} rows for '{keyword}'")
            filtered_df = filtered_df.copy()
            filtered_df["keyword"] = keyword
            all_matches = pd.concat([all_matches, filtered_df], ignore_index=True)
        else:
            print(f"⚠️ No matches found for '{keyword}'.")

    # --- 6. Save everything to one file ---
    if not all_matches.empty:
        all_matches.to_excel(output_path, index=False)
        print(f"\n📊 All matching data saved successfully to:\n{output_path}")
    else:
        print("\n❌ No matches found for any keyword.")
        return output_path


In [19]:
keywords = [
        # Secteurs d’activité
        "miroiterie",
        "métallerie",
        "menuiserie extérieure",

        # CPV simples
        
    
        
    

        # CPV détaillés avec description
        "45420000", "Travaux de menuiserie et de charpenterie",
        "45421100", "Pose de portes et de fenêtres et d'éléments accessoires",
        "45421110", "Pose d'encadrements de portes et de fenêtres",
        "45421111", "Pose d'encadrements de portes",
        "45421112", "Pose d'encadrements de fenêtres",
        "45421120", "Pose de seuils",
        "45421130", "Poses de portes et de fenêtres",
        "45421131", "Pose de portes",
        "45421132", "Pose de fenêtres",
        "45421140", "Pose de menuiseries métalliques, excepté portes et fenêtres",
        "45421141", "Travaux de cloisonnement",
        "45421142", "Installation de volets",
        "45421143", "Travaux d'installation de stores",
        "45421144", "Travaux d'installation de vélums",
        "45421145", "Travaux d'installation de volets roulants",
        "44316500", "Serrurerie",
        "98395000", "Services de serrurerie",
        "44220000", "Menuiserie pour la construction",
        "45421000", "Travaux de menuiserie",
        "45421140", "Pose de menuiseries métalliques, excepté portes et fenêtres",
        "45421150", "Travaux d'installation de menuiseries non métalliques",
        "34928200", "Clôtures",
        "34928310", "Clôtures de protection",
        "45340000", "Travaux d'installation de clôtures, de garde-corps et de dispositifs de sécurité",
        "45342000", "Pose de clôtures",
        "42416000", "Ascenseurs, skips, monte-charges, escaliers mécaniques et trottoirs roulants",
        "42416400", "Escaliers mécaniques",
        "42419500", "Pièces pour ascenseurs, skips ou escaliers mécaniques",
        "42419530", "Pièces pour escaliers mécaniques",
        "44233000", "Escaliers",
        "44423220", "Escaliers pliants",
        "45313000", "Travaux d'installation d'ascenseurs et d'escaliers mécaniques",
        "45313200", "Travaux d'installation d'escaliers mécaniques",
        "50740000", "Services de réparation et d'entretien d'escaliers mécaniques",
        "51511000", "Services d'installation de matériel de levage et de manutention, excepté ascenseurs et escaliers mécaniques",]

file_path = excel_filename
output_path = f"BOAMP_{target_date}_{datetime.now().strftime('%Y%m%d_%H%M%S')}_secteur.xlsx"


get_keword_excel(file_path, output_path, keywords)


📂 Reang Excel file...
⚠️ No matches found for 'miroiterie'.
✅ Found 7 rows for 'métallerie'
✅ Found 4 rows for 'menuiserie extérieure'
⚠️ No matches found for '45420000'.
⚠️ No matches found for 'Travaux de menuiserie et de charpenterie'.
✅ Found 1 rows for '45421100'
⚠️ No matches found for 'Pose de portes et de fenêtres et d'éléments accessoires'.
⚠️ No matches found for '45421110'.
⚠️ No matches found for 'Pose d'encadrements de portes et de fenêtres'.
⚠️ No matches found for '45421111'.
⚠️ No matches found for 'Pose d'encadrements de portes'.
⚠️ No matches found for '45421112'.
⚠️ No matches found for 'Pose d'encadrements de fenêtres'.
⚠️ No matches found for '45421120'.
⚠️ No matches found for 'Pose de seuils'.
⚠️ No matches found for '45421130'.
⚠️ No matches found for 'Poses de portes et de fenêtres'.
⚠️ No matches found for '45421131'.
⚠️ No matches found for 'Pose de portes'.
⚠️ No matches found for '45421132'.
⚠️ No matches found for 'Pose de fenêtres'.
✅ Found 2 rows for '45

In [ ]:
def delete_duplicated_rows(file_path,id_column):
# === 1️⃣ Path to your Excel file ===

    # === 2️⃣ Read the Excel file ===
    df = pd.read_excel(file_path)

    # === 3️⃣ Column name containing the ID ===
    # Change this if your ID column has a different name

    # === 4️⃣ Remove duplicate rows — keep only the first occurrence per ID ===
    df_clean = df.drop_duplicates(subset=[id_column], keep='first')

    # === 5️⃣ Save to a new Excel file ===
    df_clean.to_excel(output_path, index=False)

    print(f"✅ Done! Kept {len(df_clean)} unique rows out of {len(df)} total.")
    print(f"💾 Clean file saved to: {output_path}")


In [ ]:
id_column = 'id'
file_path = output_path
delete_duplicated_rows( file_path,id_column, )



✅ Done! Kept 25 unique rows out of 61 total.
💾 Clean file saved to: BOAMP_2025-10-31_20251102_145029_secteur.xlsx


In [ ]:
file ="D:\sorabo\sorabo\da.xlsx"
dateparution=row.get('dateparution', 'N/A')
month=f"{dateparution.month:02d}"
year=f"{dateparution.year:02d}"
idweb=row.get('idweb', 'N/A')
f"https://www.boamp.fr/telechargements/FILES/PDF/{year}/{month}/{idweb}.pdf"

In [5]:


import pandas as pd

# Read the Excel file
file = r"D:\sorabo\sorabo\da.xlsx"
df = pd.read_excel(file)

# Create list of links
links = []
for index, row in df.iterrows():
    dateparution = row.get('dateparution')
    print(dateparution)
    

        
    month = f"{dateparution.month}"
    print(month)
    year = f"{dateparution.year}"
    print(year)
    idweb = row.get('idweb', 'N/A')
    print(idweb)
    
    # Skip if idweb is not available
    if idweb == 'N/A':
        continue
        
    link = f"https://www.boamp.fr/telechargements/FILES/PDF/{year}/{month}/{idweb}.pdf"
    links.append(link)

# Print or use the links list
print(f"Generated {len(links)} links:")
for link in links:
    print(link)


2025-11-05


AttributeError: 'str' object has no attribute 'month'

In [6]:
import pandas as pd
from datetime import datetime

# Read the Excel file
file = r"D:\sorabo\sorabo\da.xlsx"
df = pd.read_excel(file)

# Create list of links
links = []
for index, row in df.iterrows():
    dateparution_str = row.get('dateparution')
    print(f"Original dateparution: {dateparution_str} (type: {type(dateparution_str)})")
    
    # Convert string to datetime object
    try:
        # Try to parse the date - adjust the format based on your actual date format
        if isinstance(dateparution_str, str):
            # Common date formats - try different ones
            date_formats = ['%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y', '%d-%m-%Y', '%Y/%m/%d']
            dateparution = None
            for fmt in date_formats:
                try:
                    dateparution = datetime.strptime(dateparution_str, fmt)
                    break
                except ValueError:
                    continue
            if dateparution is None:
                print(f"Could not parse date: {dateparution_str}")
                continue
        else:
            # If it's already a datetime object, use it directly
            dateparution = dateparution_str
        
        month = f"{dateparution.month:02d}"  # :02d to ensure 2 digits
        print(f"Month: {month}")
        year = f"{dateparution.year}"
        print(f"Year: {year}")
        idweb = row.get('idweb', 'N/A')
        print(f"ID web: {idweb}")
        
        # Skip if idweb is not available
        if idweb == 'N/A':
            continue
            
        link = f"https://www.boamp.fr/telechargements/FILES/PDF/{year}/{month}/{idweb}.pdf"
        links.append(link)
        
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        continue

# Print or use the links list
print(f"Generated {len(links)} links:")
for link in links:
    print(link)

Original dateparution: 2025-11-05 (type: <class 'str'>)
Month: 11
Year: 2025
ID web: 25-123242
Original dateparution: 2025-11-05 (type: <class 'str'>)
Month: 11
Year: 2025
ID web: 25-122618
Original dateparution: 2025-11-05 (type: <class 'str'>)
Month: 11
Year: 2025
ID web: 25-122685
Original dateparution: 2025-11-05 (type: <class 'str'>)
Month: 11
Year: 2025
ID web: 25-123427
Original dateparution: 2025-11-05 (type: <class 'str'>)
Month: 11
Year: 2025
ID web: 25-123329
Original dateparution: 2025-11-05 (type: <class 'str'>)
Month: 11
Year: 2025
ID web: 25-123097
Original dateparution: 2025-11-05 (type: <class 'str'>)
Month: 11
Year: 2025
ID web: 25-123321
Original dateparution: 2025-11-05 (type: <class 'str'>)
Month: 11
Year: 2025
ID web: 25-123088
Original dateparution: 2025-11-05 (type: <class 'str'>)
Month: 11
Year: 2025
ID web: 25-123403
Original dateparution: 2025-11-05 (type: <class 'str'>)
Month: 11
Year: 2025
ID web: 25-122444
Original dateparution: 2025-11-05 (type: <class 's

In [ ]:
import pandas as pd
from datetime import datetime

# Read the Excel file
file = r"D:\sorabo\sorabo\da.xlsx"
df = pd.read_excel(file)

# Create list of dictionaries
info_list = []

for index, row in df.iterrows():
    dateparution_str = row.get('dateparution')
    idweb = row.get('idweb', 'N/A')
    
    if idweb == 'N/A':
        continue
        
    try:
        if isinstance(dateparution_str, str):
            date_formats = ['%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y', '%d-%m-%Y', '%Y/%m/%d']
            dateparution = None
            for fmt in date_formats:
                try:
                    dateparution = datetime.strptime(dateparution_str, fmt)
                    break
                except ValueError:
                    continue
            if dateparution is None:
                continue
        else:
            dateparution = dateparution_str
        
        info_dict = {
            'year': f"{dateparution.year}",
            'month': f"{dateparution.month:02d}",
            'idweb': idweb
        }
        
        info_list.append(info_dict)
        
    except Exception:
        continue

# Display the collected information
print(f"Collected {len(info_list)} records:")
for info in info_list:
    print(info)



links = []
for item in info_list:
    link = f"https://www.boamp.fr/telechargements/FILES/PDF/{item['year']}/{item['month']}/{item['idweb']}.pdf"
    links.append(link)

print("Generated links:")
for link in links:
    print(link)

Collected 28 records:
{'year': '2025', 'month': '11', 'idweb': '25-123242'}
{'year': '2025', 'month': '11', 'idweb': '25-122618'}
{'year': '2025', 'month': '11', 'idweb': '25-122685'}
{'year': '2025', 'month': '11', 'idweb': '25-123427'}
{'year': '2025', 'month': '11', 'idweb': '25-123329'}
{'year': '2025', 'month': '11', 'idweb': '25-123097'}
{'year': '2025', 'month': '11', 'idweb': '25-123321'}
{'year': '2025', 'month': '11', 'idweb': '25-123088'}
{'year': '2025', 'month': '11', 'idweb': '25-123403'}
{'year': '2025', 'month': '11', 'idweb': '25-122444'}
{'year': '2025', 'month': '11', 'idweb': '25-123132'}
{'year': '2025', 'month': '11', 'idweb': '25-122355'}
{'year': '2025', 'month': '11', 'idweb': '25-123334'}
{'year': '2025', 'month': '11', 'idweb': '25-122350'}
{'year': '2025', 'month': '11', 'idweb': '25-123093'}
{'year': '2025', 'month': '11', 'idweb': '25-123336'}
{'year': '2025', 'month': '11', 'idweb': '25-123077'}
{'year': '2025', 'month': '11', 'idweb': '25-123177'}
{'year

In [8]:
# Method 1: Simple for loop to create links
links = []
for item in info_list:
    link = f"https://www.boamp.fr/telechargements/FILES/PDF/{item['year']}/{item['month']}/{item['idweb']}.pdf"
    links.append(link)

print("Generated links:")
for link in links:
    print(link)

Generated links:
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123242.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-122618.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-122685.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123427.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123329.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123097.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123321.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123088.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123403.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-122444.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123132.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-122355.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123334.pdf
https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-122350.pdf
https://www.boamp

In [9]:
import requests
import tempfile
import os
import subprocess

links = []
for item in info_list:
    link = f"https://www.boamp.fr/telechargements/FILES/PDF/{item['year']}/{item['month']}/{item['idweb']}.pdf"
    links.append(link)

print("Downloading and printing PDFs...")

for i, link in enumerate(links):
    try:
        print(f"Processing {i+1}/{len(links)}: {link}")
        
        # Download the PDF
        response = requests.get(link, timeout=30)
        response.raise_for_status()
        
        # Create a temporary file
        with tempfile.NamedTemporaryFile(suffix='.pdf', delete=False) as temp_file:
            temp_file.write(response.content)
            temp_path = temp_file.name
        
        # Print the PDF (Windows)
        if os.name == 'nt':  # Windows
            os.startfile(temp_path, "print")
            print(f"  Sent to printer: {link}")
        else:  # Linux/Mac
            # For Linux: lpr command
            # For Mac: use 'lp' command
            subprocess.run(['lp', temp_path])
            print(f"  Sent to printer: {link}")
        
        # Clean up (you might want to remove this if you want to keep the files)
        os.unlink(temp_path)
        
    except requests.exceptions.RequestException as e:
        print(f"  Error downloading {link}: {e}")
    except Exception as e:
        print(f"  Error printing {link}: {e}")

Processing 1/28: https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123242.pdf
  Error printing https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123242.pdf: [WinError 1155] No application is associated with the specified file for this operation: 'C:\\Users\\faycal\\AppData\\Local\\Temp\\tmpgkxukw5t.pdf'
Processing 2/28: https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-122618.pdf
  Error printing https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-122618.pdf: [WinError 1155] No application is associated with the specified file for this operation: 'C:\\Users\\faycal\\AppData\\Local\\Temp\\tmpkw_7xvon.pdf'
Processing 3/28: https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-122685.pdf
  Error printing https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-122685.pdf: [WinError 1155] No application is associated with the specified file for this operation: 'C:\\Users\\faycal\\AppData\\Local\\Temp\\tmperhewaga.pdf'
Processing 4/28: https://www.boamp.fr

In [10]:
pip install PyPDF2


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\sorabo\sorabo\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [12]:
import requests
import tempfile
import os
import PyPDF2

links = []
for item in info_list:
    link = f"https://www.boamp.fr/telechargements/FILES/PDF/{item['year']}/{item['month']}/{item['idweb']}.pdf"
    links.append(link)

print("Downloading and extracting text from PDFs using PyPDF2...")

for i, link in enumerate(links):
    try:
        print(f"\n{'='*50}")
        print(f"Processing {i+1}/{len(links)}: {link}")
        
        # Download the PDF
        response = requests.get(link, timeout=30)
        response.raise_for_status()
        
        # Save to temporary file
        with tempfile.NamedTemporaryFile(suffix='.pdf', delete=False) as temp_file:
            temp_file.write(response.content)
            temp_path = temp_file.name
        
        # Extract text using PyPDF2
        with open(temp_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            
            print(f"Number of pages: {len(pdf_reader.pages)}")
            
            # Extract text from each page
            full_text = ""
            for page_num, page in enumerate(pdf_reader.pages):
                page_text = page.extract_text()
                full_text += f"\n--- Page {page_num + 1} ---\n{page_text}\n"
            
            # Print the extracted text
            print(f"Extracted text from {link}:")
            print(full_text)  # Limit output
        
        # Clean up
        os.unlink(temp_path)
        
    except Exception as e:
        print(f"Error processing {link}: {e}")


Processing 1/28: https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123242.pdf
Number of pages: 3
Extracted text from https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123242.pdf:

--- Page 1 ---
1/3
Avis de marché
Attention : les informations contenues dans l'extrait PDF peuvent dans certains cas ne pas présenter le texte 
intégral de l'annonce. Les extraits PDF des annonces du BOAMP ne constituent pas le format officiel, pour 
consulter le texte intégral au format officiel du présent avis, cliquez sur  https://www.boamp.fr/pages/avis/?
q=idweb:25-123242
Département(s) de publication : 73
 Annonce n° 25-123242
Travaux
Section 1 - Identification de l'acheteur
Ville d'Aix les Bains Nom complet de l'acheteur : 
SIRET Type de Numéro national d'indentification : 
21730008600014 N° National d'identification : 
Aix les bains Ville : 
73103 Code postal : 
 Non Groupement de commandes :
73 Département(s) de publication : 
Section 2 - Communication
 Lien vers le profil d'acheteu

In [ ]:
import pandas as pd
from datetime import datetime

# Read the Excel file
file = r"D:\sorabo\sorabo\da.xlsx"
df = pd.read_excel(file)

# Create list of dictionaries
info_list = []

for index, row in df.iterrows():
    dateparution_str = row.get('dateparution')
    idweb = row.get('idweb', 'N/A')
    
    if idweb == 'N/A':
        continue
        
    try:
        if isinstance(dateparution_str, str):
            date_formats = ['%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y', '%d-%m-%Y', '%Y/%m/%d']
            dateparution = None
            for fmt in date_formats:
                try:
                    dateparution = datetime.strptime(dateparution_str, fmt)
                    break
                except ValueError:
                    continue
            if dateparution is None:
                continue
        else:
            dateparution = dateparution_str
        
        info_dict = {
            'year': f"{dateparution.year}",
            'month': f"{dateparution.month:02d}",
            'idweb': idweb
        }
        
        info_list.append(info_dict)
        
    except Exception:
        continue

# Display the collected information
print(f"Collected {len(info_list)} records:")
for info in info_list:
    print(info)



links = []
for item in info_list:
    link = f"https://www.boamp.fr/telechargements/FILES/PDF/{item['year']}/{item['month']}/{item['idweb']}.pdf"
    links.append(link)

print("Generated links:")
for link in links:
    print(link)



import requests
import tempfile
import os
import PyPDF2

links = []
for item in info_list:
    link = f"https://www.boamp.fr/telechargements/FILES/PDF/{item['year']}/{item['month']}/{item['idweb']}.pdf"
    links.append(link)

print("Downloading and extracting text from PDFs using PyPDF2...")

for i, link in enumerate(links):
    try:
        print(f"\n{'='*50}")
        print(f"Processing {i+1}/{len(links)}: {link}")
        
        # Download the PDF
        response = requests.get(link, timeout=30)
        response.raise_for_status()
        
        # Save to temporary file
        with tempfile.NamedTemporaryFile(suffix='.pdf', delete=False) as temp_file:
            temp_file.write(response.content)
            temp_path = temp_file.name
        
        # Extract text using PyPDF2
        with open(temp_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            
            print(f"Number of pages: {len(pdf_reader.pages)}")
            
            # Extract text from each page
            full_text = ""
            for page_num, page in enumerate(pdf_reader.pages):
                page_text = page.extract_text()
                full_text += f"\n--- Page {page_num + 1} ---\n{page_text}\n"
            
            # Print the extracted text
            print(f"Extracted text from {link}:")
            print(full_text)  # Limit output
        
        # Clean up
        os.unlink(temp_path)
        
    except Exception as e:
        print(f"Error processing {link}: {e}")

In [13]:
import pandas as pd
from datetime import datetime
import requests
import tempfile
import os
import PyPDF2
import time

# Read the Excel file
file = r"D:\sorabo\sorabo\da.xlsx"
df = pd.read_excel(file)

# Create new columns for links and PDF content
df['generated_link'] = ""
df['pdf_content'] = ""

print(f"Processing {len(df)} records...")

for index, row in df.iterrows():
    dateparution_str = row.get('dateparution')
    idweb = row.get('idweb', 'N/A')
    
    if idweb == 'N/A':
        continue
        
    try:
        # Parse date
        if isinstance(dateparution_str, str):
            date_formats = ['%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y', '%d-%m-%Y', '%Y/%m/%d']
            dateparution = None
            for fmt in date_formats:
                try:
                    dateparution = datetime.strptime(dateparution_str, fmt)
                    break
                except ValueError:
                    continue
            if dateparution is None:
                continue
        else:
            dateparution = dateparution_str
        
        # Generate link
        link = f"https://www.boamp.fr/telechargements/FILES/PDF/{dateparution.year}/{dateparution.month:02d}/{idweb}.pdf"
        
        # Add link to DataFrame
        df.at[index, 'generated_link'] = link
        
        # Download and extract PDF content
        print(f"Processing {index+1}/{len(df)}: {link}")
        
        try:
            # Download the PDF
            response = requests.get(link, timeout=30)
            response.raise_for_status()
            
            # Save to temporary file
            with tempfile.NamedTemporaryFile(suffix='.pdf', delete=False) as temp_file:
                temp_file.write(response.content)
                temp_path = temp_file.name
            
            # Extract text using PyPDF2
            with open(temp_path, 'rb') as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                
                # Extract text from each page
                full_text = ""
                for page_num, page in enumerate(pdf_reader.pages):
                    page_text = page.extract_text()
                    full_text += f"Page {page_num + 1}:\n{page_text}\n\n"
                
                # Add PDF content to DataFrame
                df.at[index, 'pdf_content'] = full_text
            
            # Clean up
            os.unlink(temp_path)
            
            print(f"✓ Successfully extracted {len(pdf_reader.pages)} pages")
            
        except Exception as e:
            error_msg = f"Error processing PDF: {str(e)}"
            df.at[index, 'pdf_content'] = error_msg
            print(f"✗ {error_msg}")
        
        # Add a small delay to be respectful to the server
        time.sleep(1)
        
    except Exception as e:
        error_msg = f"Error processing row: {str(e)}"
        df.at[index, 'pdf_content'] = error_msg
        print(f"✗ {error_msg}")
        continue

# Display the updated DataFrame with new columns
print("\n" + "="*80)
print("UPDATED DATAFRAME WITH LINKS AND PDF CONTENT:")
print("="*80)

# Display relevant columns to see the results
columns_to_display = ['idweb', 'objet', 'generated_link', 'pdf_content']
print(df[columns_to_display].head())

# Show sample of PDF content (first 200 characters)
print("\n" + "="*80)
print("SAMPLE OF PDF CONTENT (first 200 characters):")
print("="*80)
for idx, row in df.head().iterrows():
    if row['pdf_content'] and len(row['pdf_content']) > 0:
        content_preview = row['pdf_content'][:200] + "..." if len(row['pdf_content']) > 200 else row['pdf_content']
        print(f"\n{row['idweb']}: {content_preview}")

# Save the updated DataFrame back to Excel if needed
output_file = r"D:\sorabo\sorabo\da_with_pdf_content.xlsx"
df.to_excel(output_file, index=False)
print(f"\nUpdated DataFrame saved to: {output_file}")

Processing 28 records...
Processing 1/28: https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123242.pdf
✓ Successfully extracted 3 pages
Processing 2/28: https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-122618.pdf
✓ Successfully extracted 31 pages
Processing 3/28: https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-122685.pdf
✓ Successfully extracted 17 pages
Processing 4/28: https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123427.pdf
✓ Successfully extracted 2 pages
Processing 5/28: https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123329.pdf
✓ Successfully extracted 3 pages
Processing 6/28: https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123097.pdf
✓ Successfully extracted 1 pages
Processing 7/28: https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123321.pdf
✓ Successfully extracted 2 pages
Processing 8/28: https://www.boamp.fr/telechargements/FILES/PDF/2025/11/25-123088.pdf
✓ Successfully extracted 3 pages
Processing 9/28: http

In [1]:
import PyPDF2
import re

def search_keywords_and_find_lot(pdf_path, keywords):
    """
    Search for keywords in a PDF and find the lot numbers that appear before them
    """
    try:
        # Open the PDF file
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            results = []
            
            # Iterate through each page
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                # Search for each keyword
                for keyword in keywords:
                    # Find all occurrences of the keyword
                    keyword_matches = re.finditer(re.escape(keyword), text, re.IGNORECASE)
                    
                    for match in keyword_matches:
                        keyword_position = match.start()
                        
                        # Extract text before the keyword (look back up to 200 characters)
                        text_before = text[max(0, keyword_position - 200):keyword_position]
                        
                        # Search for "lot" or "LOT" followed by a number
                        lot_pattern = r'(lot|LOT)\s*[:\-\s]*\s*(\d+[-\d]*)'
                        lot_matches = re.findall(lot_pattern, text_before, re.IGNORECASE)
                        
                        if lot_matches:
                            # Get the last lot found before the keyword (most recent)
                            last_lot = lot_matches[-1]
                            lot_number = last_lot[1]
                            
                            results.append({
                                'page': page_num + 1,
                                'keyword': keyword,
                                'lot_number': lot_number,
                                'position': keyword_position
                            })
                            
                            print(f"Found: lot-{lot_number} (before '{keyword}' on page {page_num + 1})")
            
            return results
            
    except FileNotFoundError:
        print(f"Error: File not found at {pdf_path}")
        return []
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return []

# Main execution
if __name__ == "__main__":
    pdf_path = r"D:\sorabo\sorabo\pdf\25-124195.pdf"
    
    # Keywords to search for
    keywords = [
        "Serrurerie",
        "45420000",
        "45421141", 
        "44316500",
        "45421000"
    ]
    
    print(f"Searching for keywords in: {pdf_path}")
    print("Keywords:", ", ".join(keywords))
    print("-" * 50)
    
    results = search_keywords_and_find_lot(pdf_path, keywords)
    
    if not results:
        print("No matching keywords with preceding lot numbers found.")
    else:
        print("\n" + "=" * 50)
        print("SUMMARY:")
        print("=" * 50)
        for result in results:
            print(f"lot-{result['lot_number']}")

Searching for keywords in: D:\sorabo\sorabo\pdf\25-124195.pdf
Keywords: Serrurerie, 45420000, 45421141, 44316500, 45421000
--------------------------------------------------
Found: lot-0010 (before 'Serrurerie' on page 22)
Found: lot-0010 (before 'Serrurerie' on page 22)

SUMMARY:
lot-0010
lot-0010


In [ ]:
import PyPDF2
import re

def detailed_search(pdf_path, keywords):
    """
    More detailed search with additional context
    """
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                print(f"\n=== Page {page_num + 1} ===")
                
                for keyword in keywords:
                    if keyword.lower() in text.lower():
                        print(f"\nFound keyword: '{keyword}'")
                        
                        # Find context around the keyword
                        keyword_index = text.lower().find(keyword.lower())
                        start_context = max(0, keyword_index - 100)
                        end_context = min(len(text), keyword_index + len(keyword) + 100)
                        
                        context = text[start_context:end_context]
                        print(f"Context: ...{context}...")
                        
                        # Look for lot numbers in the preceding text
                        preceding_text = text[max(0, keyword_index - 300):keyword_index]
                        lot_matches = re.findall(r'(lot|LOT)\s*[:\-\s]*\s*(\d+[-\d]*)', preceding_text, re.IGNORECASE)
                        
                        if lot_matches:
                            for lot_match in lot_matches:
                                print(f"→ Found lot number: {lot_match[1]}")
                        else:
                            print("→ No lot number found before this keyword")
    
    except Exception as e:
        print(f"Error: {e}")

# Run both searches
if __name__ == "__main__":
    pdf_path = r"D:\sorabo\sorabo\pdf\25-124195.pdf"
    keywords = ["Serrurerie", "45420000", "45421141", "44316500", "45421000"]
    
    print("=== BASIC SEARCH ===")
    results = search_keywords_and_find_lot(pdf_path, keywords)
    
    print("\n=== DETAILED SEARCH ===")
    detailed_search(pdf_path, keywords)

In [3]:
import PyPDF2
import re

def search_keywords_and_find_lot(pdf_path, keywords):
    """
    Search for keywords in a PDF and find ALL lot numbers that appear before them
    """
    try:
        # Open the PDF file
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            results = []
            
            # Iterate through each page
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                print(f"\n=== Searching Page {page_num + 1} ===")
                
                # Search for each keyword
                for keyword in keywords:
                    # Find all occurrences of the keyword
                    keyword_matches = list(re.finditer(re.escape(keyword), text, re.IGNORECASE))
                    
                    if keyword_matches:
                        print(f"Found '{keyword}' {len(keyword_matches)} time(s) on page {page_num + 1}")
                    
                    for match in keyword_matches:
                        keyword_position = match.start()
                        
                        # Extract more text before the keyword (look back up to 500 characters)
                        text_before = text[max(0, keyword_position - 500):keyword_position]
                        
                        # Improved lot pattern to catch more formats
                        lot_patterns = [
                            r'(lot|LOT)\s*[:\-\s]*\s*(\d+[-\w]*)',  # lot: 123, LOT-456, lot 789
                            r'(Lot\s*\d+)',  # Lot 123
                            r'(lot\s*\d+)',  # lot 123
                            r'\b(\d+)\s*-\s*Lot',  # 123 - Lot
                            r'\b(LOT\s*[A-Z]*\d+)',  # LOT A123, LOT 456
                        ]
                        
                        all_lot_matches = []
                        
                        for pattern in lot_patterns:
                            matches = re.findall(pattern, text_before, re.IGNORECASE)
                            for match in matches:
                                if isinstance(match, tuple):
                                    # For patterns that capture groups
                                    lot_number = match[1] if len(match) > 1 else match[0]
                                else:
                                    # For patterns that capture directly
                                    lot_number = match
                                
                                # Clean up the lot number
                                lot_number = re.sub(r'^(lot|LOT)\s*', '', lot_number, flags=re.IGNORECASE)
                                lot_number = lot_number.strip(' :-\t')
                                
                                if lot_number and lot_number not in [lm[0] for lm in all_lot_matches]:
                                    all_lot_matches.append((lot_number, pattern))
                        
                        # Remove duplicates while preserving order
                        unique_lots = []
                        seen = set()
                        for lot_num, pattern in all_lot_matches:
                            if lot_num not in seen:
                                seen.add(lot_num)
                                unique_lots.append(lot_num)
                        
                        if unique_lots:
                            for lot_number in unique_lots:
                                results.append({
                                    'page': page_num + 1,
                                    'keyword': keyword,
                                    'lot_number': lot_number,
                                    'position': keyword_position
                                })
                                print(f"Found: lot-{lot_number} (before '{keyword}' on page {page_num + 1})")
                        else:
                            print(f"Keyword '{keyword}' found on page {page_num + 1}, but no lot number detected before it")
                            # Print context for debugging
                            context = text_before[-100:]  # Last 100 chars before keyword
                            print(f"Context before keyword: ...{context}")
            
            return results
            
    except FileNotFoundError:
        print(f"Error: File not found at {pdf_path}")
        return []
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return []

def alternative_search_method(pdf_path, keywords):
    """
    Alternative method: Extract all lot numbers and keywords, then find associations
    """
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            print("\n" + "="*60)
            print("ALTERNATIVE SEARCH METHOD")
            print("="*60)
            
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                # Find all lot numbers with broader pattern
                lot_pattern = r'\b(?:lot|LOT|Lot)\s*[:\-]?\s*([A-Z0-9][\w\-]*)\b'
                lot_matches = re.finditer(lot_pattern, text, re.IGNORECASE)
                
                lots_with_positions = []
                for match in lot_matches:
                    lot_number = match.group(1)
                    lots_with_positions.append({
                        'number': lot_number,
                        'position': match.start(),
                        'context': text[max(0, match.start()-20):match.end()+20]
                    })
                
                # Find all keywords with positions
                keywords_with_positions = []
                for keyword in keywords:
                    keyword_matches = re.finditer(re.escape(keyword), text, re.IGNORECASE)
                    for match in keyword_matches:
                        keywords_with_positions.append({
                            'keyword': keyword,
                            'position': match.start(),
                            'context': text[max(0, match.start()-20):match.end()+20]
                        })
                
                # Sort both lists by position
                lots_with_positions.sort(key=lambda x: x['position'])
                keywords_with_positions.sort(key=lambda x: x['position'])
                
                # For each keyword, find the closest preceding lot
                for kw in keywords_with_positions:
                    preceding_lots = [lot for lot in lots_with_positions if lot['position'] < kw['position']]
                    
                    if preceding_lots:
                        # Get all lots that appear before this keyword
                        for lot in preceding_lots:
                            print(f"Alternative found: lot-{lot['number']} (before '{kw['keyword']}' on page {page_num + 1})")
                    else:
                        print(f"Keyword '{kw['keyword']}' found on page {page_num + 1} but no preceding lot numbers")
    
    except Exception as e:
        print(f"Error in alternative search: {e}")

# Main execution
if __name__ == "__main__":
    pdf_path = r"D:\sorabo\sorabo\pdf\25-124195.pdf"
    
    # Keywords to search for
    keywords = [
        "Serrurerie",
        "45420000",
        "45421141", 
        "44316500",
        "45421000"
    ]
    
    print(f"Searching for keywords in: {pdf_path}")
    print("Keywords:", ", ".join(keywords))
    print("=" * 60)
    
    # Method 1: Improved search
    results = search_keywords_and_find_lot(pdf_path, keywords)
    
    # Method 2: Alternative approach
    alternative_search_method(pdf_path, keywords)
    
    # Final summary
    if results:
        print("\n" + "=" * 60)
        print("FINAL SUMMARY (Method 1):")
        print("=" * 60)
        unique_lots = set()
        for result in results:
            unique_lots.add(f"lot-{result['lot_number']}")
        
        for lot in sorted(unique_lots):
            print(lot)
    else:
        print("\nNo results found with Method 1.")

Searching for keywords in: D:\sorabo\sorabo\pdf\25-124195.pdf
Keywords: Serrurerie, 45420000, 45421141, 44316500, 45421000

=== Searching Page 1 ===

=== Searching Page 2 ===

=== Searching Page 3 ===

=== Searching Page 4 ===

=== Searching Page 5 ===

=== Searching Page 6 ===

=== Searching Page 7 ===

=== Searching Page 8 ===

=== Searching Page 9 ===

=== Searching Page 10 ===

=== Searching Page 11 ===
Found '45420000' 1 time(s) on page 11
Found: lot-0005 (before '45420000' on page 11)

=== Searching Page 12 ===

=== Searching Page 13 ===

=== Searching Page 14 ===

=== Searching Page 15 ===

=== Searching Page 16 ===

=== Searching Page 17 ===

=== Searching Page 18 ===
Found '45421141' 1 time(s) on page 18
Keyword '45421141' found on page 18, but no lot number detected before it
Context before keyword: ...principale (cpv ):45262800 Travaux d'extension de bâtiments
     Nomenclature supplémentaire (cpv ):

=== Searching Page 19 ===

=== Searching Page 20 ===
Found '45421000' 1 ti

In [20]:
import PyPDF2
import re

def search_keywords_and_find_lot(pdf_path, keywords):
    """
    Search for keywords in a PDF and find ALL lot numbers that appear before them
    """
    try:
        # Open the PDF file
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            results = []
            
            # Iterate through each page
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                
                # Search for each keyword
                for keyword in keywords:
                    # Find all occurrences of the keyword
                    keyword_matches = list(re.finditer(re.escape(keyword), text, re.IGNORECASE))
                    
                    
                    for match in keyword_matches:
                        keyword_position = match.start()
                        
                        # Extract more text before the keyword (look back up to 500 characters)
                        text_before = text[max(0, keyword_position - 500):keyword_position]
                        
                        # Improved lot pattern to catch more formats
                        lot_patterns = [
                            r'(lot|LOT)\s*[:\-\s]*\s*(\d+[-\w]*)',  # lot: 123, LOT-456, lot 789
                            r'(Lot\s*\d+)',  # Lot 123
                            r'(lot\s*\d+)',  # lot 123
                            r'\b(\d+)\s*-\s*Lot',  # 123 - Lot
                            r'\b(LOT\s*[A-Z]*\d+)',  # LOT A123, LOT 456
                        ]
                        
                        all_lot_matches = []
                        
                        for pattern in lot_patterns:
                            matches = re.findall(pattern, text_before, re.IGNORECASE)
                            for match in matches:
                                if isinstance(match, tuple):
                                    # For patterns that capture groups
                                    lot_number = match[1] if len(match) > 1 else match[0]
                                else:
                                    # For patterns that capture directly
                                    lot_number = match
                                
                                # Clean up the lot number
                                lot_number = re.sub(r'^(lot|LOT)\s*', '', lot_number, flags=re.IGNORECASE)
                                lot_number = lot_number.strip(' :-\t')
                                
                                if lot_number and lot_number not in [lm[0] for lm in all_lot_matches]:
                                    all_lot_matches.append((lot_number, pattern))
                        
                        # Remove duplicates while preserving order
                        unique_lots = []
                        seen = set()
                        for lot_num, pattern in all_lot_matches:
                            if lot_num not in seen:
                                seen.add(lot_num)
                                unique_lots.append(lot_num)
                        
                        if unique_lots:
                            for lot_number in unique_lots:
                                results.append({
                                    'page': page_num + 1,
                                    'keyword': keyword,
                                    'lot_number': lot_number,
                                    'position': keyword_position
                                })
                                print(f"Found: lot-{lot_number} (before '{keyword}' on page {page_num + 1})")
                        else:
                            # Print context for debugging
                            context = text_before[-100:]  # Last 100 chars before keyword
            
            return results
            
    except FileNotFoundError:
        return []
    except Exception as e:
        return []



# Main execution
if __name__ == "__main__":
    pdf_path = r"D:\sorabo\sorabo\pdf\25-124195.pdf"
    
    # Keywords to search for
    keywords = [
        "Serrurerie",
        "45420000",
        "45421141", 
        "44316500",
        "45421000"
    ]
    
   
    
    # Method 1: Improved search
    results = search_keywords_and_find_lot(pdf_path, keywords)
    
    # Method 2: Alternative approach
    
    # Final summary
    if results:

        unique_lots = set()
        for result in results:
            unique_lots.add(f"lot-{result['lot_number']}")
        
        for lot in sorted(unique_lots):
            print(lot)
    else:
        print("\nNo results found with Method 1.")

Found: lot-0005 (before '45420000' on page 11)
Found: lot-0009 (before '45421000' on page 20)
Found: lot-0010 (before 'Serrurerie' on page 22)
Found: lot-0010 (before 'Serrurerie' on page 22)
Found: lot-0010 (before 'Serrurerie' on page 22)
Found: lot-0010 (before '44316500' on page 22)
lot-0005
lot-0009
lot-0010


In [ ]:
import PyPDF2
import re

def search_keywords_and_find_lot(pdf_path, keywords):
    """
    Search for keywords in a PDF and find ALL lot numbers that appear before them
    """
    try:
        # Open the PDF file
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            results = []
            
            # Iterate through each page
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                
                # Search for each keyword
                for keyword in keywords:
                    # Find all occurrences of the keyword
                    keyword_matches = list(re.finditer(re.escape(keyword), text, re.IGNORECASE))
                    
                    
                    for match in keyword_matches:
                        keyword_position = match.start()
                        
                        # Extract more text before the keyword (look back up to 500 characters)
                        text_before = text[max(0, keyword_position - 50):keyword_position]
                        
                        # Improved lot pattern to catch more formats
                        lot_patterns = ["visites ","visite"
                        ]
                        
                        all_lot_matches = []
                        
                        for pattern in lot_patterns:
                            matches = re.findall(pattern, text_before, re.IGNORECASE)
                            for match in matches:
                                if isinstance(match, tuple):
                                    # For patterns that capture groups
                                    lot_number = match[1] if len(match) > 1 else match[0]
                                else:
                                    # For patterns that capture directly
                                    lot_number = match
                                
                                # Clean up the lot number
                                lot_number = re.sub(r'^(lot|LOT)\s*', '', lot_number, flags=re.IGNORECASE)
                                lot_number = lot_number.strip(' :-\t')
                                
                                if lot_number and lot_number not in [lm[0] for lm in all_lot_matches]:
                                    all_lot_matches.append((lot_number, pattern))
                        
                        # Remove duplicates while preserving order
                        unique_lots = []
                        seen = set()
                        for lot_num, pattern in all_lot_matches:
                            if lot_num not in seen:
                                seen.add(lot_num)
                                unique_lots.append(lot_num)
                        
                        if unique_lots:
                            for lot_number in unique_lots:
                                results.append({
                                    'page': page_num + 1,
                                    'keyword': keyword,
                                    'lot_number': lot_number,
                                    'position': keyword_position
                                })
                                print(f"Found: lot-{lot_number} (before '{keyword}' on page {page_num + 1})")
                        else:
                            # Print context for debugging
                            context = text_before[-100:]  # Last 100 chars before keyword
            
            return results
            
    except FileNotFoundError:
        return []
    except Exception as e:
        return []



# Main execution
if __name__ == "__main__":
    pdf_path = r"D:\sorabo\sorabo\pdf\25-124268.pdf"
    
    # Keywords to search for
    keywords = [
             "obligatoires"
    ]
    
   
    
    # Method 1: Improved search
    results = search_keywords_and_find_lot(pdf_path, keywords)
    
    # Method 2: Alternative approach
    
    # Final summary
    if results:

        unique_lots = set()
        for result in results:
            unique_lots.add(f"lot-{result['lot_number']}")
        
        for lot in sorted(unique_lots):
            print(lot)
    else:
        print("No results found with Method 1.")


No results found with Method 1.


In [17]:
import PyPDF2
import re

def search_keywords_and_find_lot(pdf_path, keywords):
    """
    Search for keywords in a PDF and find ALL lot numbers that appear before them,
    then extract the complete text after each lot number
    """
    try:
        # Open the PDF file
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            results = []
            
            # Iterate through each page
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                # Search for each keyword
                for keyword in keywords:
                    # Find all occurrences of the keyword
                    keyword_matches = list(re.finditer(re.escape(keyword), text, re.IGNORECASE))
                    
                    for match in keyword_matches:
                        keyword_position = match.start()
                        
                        # Extract more text before the keyword (look back up to 500 characters)
                        text_before = text[max(0, keyword_position - 500):keyword_position]
                        
                        # Improved lot pattern to catch more formats
                        lot_patterns = [
                            r'(lot|LOT)\s*[:\-\s]*\s*(\d+[-\w]*)',  # lot: 123, LOT-456, lot 789
                            r'(Lot\s*\d+)',  # Lot 123
                            r'(lot\s*\d+)',  # lot 123
                            r'\b(\d+)\s*-\s*Lot',  # 123 - Lot
                            r'\b(LOT\s*[A-Z]*\d+)',  # LOT A123, LOT 456
                        ]
                        
                        all_lot_matches = []
                        
                        for pattern in lot_patterns:
                            matches = re.finditer(pattern, text_before, re.IGNORECASE)
                            for match_obj in matches:
                                if isinstance(match_obj.groups(), tuple):
                                    # For patterns that capture groups
                                    if len(match_obj.groups()) > 1:
                                        lot_number = match_obj.group(2)
                                    else:
                                        lot_number = match_obj.group(1)
                                else:
                                    lot_number = match_obj.group(0)
                                
                                # Clean up the lot number
                                lot_number = re.sub(r'^(lot|LOT)\s*', '', lot_number, flags=re.IGNORECASE)
                                lot_number = lot_number.strip(' :-\t')
                                
                                # Get the position of this lot number in the full text
                                lot_abs_position = keyword_position - 500 + match_obj.start()
                                
                                if lot_number:
                                    all_lot_matches.append((lot_number, lot_abs_position))
                        
                        # Remove duplicates while preserving order
                        unique_lots = []
                        seen = set()
                        for lot_num, pos in all_lot_matches:
                            if lot_num not in seen:
                                seen.add(lot_num)
                                unique_lots.append((lot_num, pos))
                        
                        # Extract full text after each lot number
                        for lot_number, lot_position in unique_lots:
                            # Extract text starting from the lot number position
                            text_after_lot = text[lot_position:]
                            
                            # Try to extract a meaningful section (title + description)
                            # Look for natural boundaries: next lot, page break, or significant section change
                            section_text = extract_complete_section(text_after_lot, lot_number)
                            
                            results.append({
                                'page': page_num + 1,
                                'keyword': keyword,
                                'lot_number': lot_number,
                                'position': keyword_position,
                                'full_text': section_text,
                                'lot_position': lot_position
                            })
            
            return results
            
    except FileNotFoundError:
        print(f"File not found: {pdf_path}")
        return []
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return []

def extract_complete_section(text_after_lot, lot_number):
    """
    Extract a complete section of text after a lot number
    """
    # Look for natural section endings
    end_patterns = [
        r'\n\s*(lot|LOT)\s*[:\-\s]*\s*\d',  # Next lot number
        r'\n\s*\d+\s*\.',  # Next numbered item
        r'\n\s*[A-Z][A-Z\s]+\n',  # ALL CAPS heading
        r'\n\s*[IVX]+\.',  # Roman numerals
    ]
    
    # First, try to find a logical end point
    end_position = len(text_after_lot)
    for pattern in end_patterns:
        match = re.search(pattern, text_after_lot)
        if match and match.start() < end_position:
            end_position = match.start()
    
    # If no natural end found, take a reasonable chunk (first 2000 characters)
    if end_position == len(text_after_lot):
        end_position = min(2000, len(text_after_lot))
    
    section = text_after_lot[:end_position].strip()
    
    # Clean up the text
    section = re.sub(r'\n+', ' ', section)  # Replace multiple newlines with spaces
    section = re.sub(r'\s+', ' ', section)  # Replace multiple spaces with single space
    
    return section

def main():
    pdf_path = r"D:\sorabo\sorabo\pdf\25-124195.pdf"
    
    # Keywords to search for
    keywords = [
        "Serrurerie",
        "45420000",
        "45421141", 
        "44316500",
        "45421000"
    ]
    
    print(f"Searching PDF: {pdf_path}")
    print(f"Keywords: {keywords}")
    print("=" * 80)
    
    results = search_keywords_and_find_lot(pdf_path, keywords)
    
    if results:
        # Group by lot number for better organization
        lot_groups = {}
        for result in results:
            lot_key = f"lot-{result['lot_number']}"
            if lot_key not in lot_groups:
                lot_groups[lot_key] = []
            lot_groups[lot_key].append(result)
        
        # Print results organized by lot number
        for lot_number, occurrences in lot_groups.items():
            print(f"\n{'='*80}")
            print(f"📋 LOT: {lot_number}")
            print(f"{'='*80}")
            
            for i, occurrence in enumerate(occurrences, 1):
                print(f"\n📍 Occurrence {i}:")
                print(f"   Page: {occurrence['page']}")
                print(f"   Found via keyword: '{occurrence['keyword']}'")
                print(f"\n   📝 Complete Text:")
                print(f"   {'─'*50}")
                # Print the text with proper formatting
                text_lines = occurrence['full_text'].split('. ')
                for line in text_lines:
                    if line.strip():
                        print(f"   {line.strip()}.")
                print(f"   {'─'*50}")
        
        print(f"\n🎯 Summary: Found {len(lot_groups)} unique lot numbers")
        
    else:
        print("\n❌ No results found.")

# Alternative function for better text extraction


if __name__ == "__main__":
    main()
    


Searching PDF: D:\sorabo\sorabo\pdf\25-124195.pdf
Keywords: ['Serrurerie', '45420000', '45421141', '44316500', '45421000']

📋 LOT: lot-0005

📍 Occurrence 1:
   Page: 11
   Found via keyword: '45420000'

   📝 Complete Text:
   ──────────────────────────────────────────────────
   LOT-0005 Titre :MENUISERIE EXTERIEURE ALUMINIUM THERMOLAQUE - VITRERIE - PROTECTION SOLAIRE- OCCULTATIONS Description :MENUISERIE EXTERIEURE ALUMINIUM THERMOLAQUE - VITRERIE - PROTECTION SOLAIRE- OCCULTATIONS Identifiant interne :5.
   ──────────────────────────────────────────────────

📋 LOT: lot-0009

📍 Occurrence 1:
   Page: 20
   Found via keyword: '45421000'

   📝 Complete Text:
   ──────────────────────────────────────────────────
   LOT-0009 Titre :MENUISERIE INTERIEURE BOIS - MOBILIERS - SIGNALETIQUE Description :MENUISERIE INTERIEURE BOIS - MOBILIERS - SIGNALETIQUE Identifiant interne :9.
   ──────────────────────────────────────────────────

📋 LOT: lot-0010

📍 Occurrence 1:
   Page: 22
   Found via ke

In [18]:
import PyPDF2
import re

def search_keywords_and_find_lot(pdf_path, keywords):
    """
    Search for keywords in a PDF and find ALL lot numbers that appear before them,
    then extract the complete text after each lot number
    """
    try:
        # Open the PDF file
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            # Dictionary to store unique lots with their information
            unique_lots = {}
            
            # Iterate through each page
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                # Search for each keyword
                for keyword in keywords:
                    # Find all occurrences of the keyword
                    keyword_matches = list(re.finditer(re.escape(keyword), text, re.IGNORECASE))
                    
                    for match in keyword_matches:
                        keyword_position = match.start()
                        
                        # Extract more text before the keyword (look back up to 500 characters)
                        text_before = text[max(0, keyword_position - 500):keyword_position]
                        
                        # Improved lot pattern to catch more formats
                        lot_patterns = [
                            r'(lot|LOT)\s*[:\-\s]*\s*(\d+[-\w]*)',  # lot: 123, LOT-456, lot 789
                            r'(Lot\s*\d+)',  # Lot 123
                            r'(lot\s*\d+)',  # lot 123
                            r'\b(\d+)\s*-\s*Lot',  # 123 - Lot
                            r'\b(LOT\s*[A-Z]*\d+)',  # LOT A123, LOT 456
                        ]
                        
                        all_lot_matches = []
                        
                        for pattern in lot_patterns:
                            matches = re.finditer(pattern, text_before, re.IGNORECASE)
                            for match_obj in matches:
                                if isinstance(match_obj.groups(), tuple):
                                    # For patterns that capture groups
                                    if len(match_obj.groups()) > 1:
                                        lot_number = match_obj.group(2)
                                    else:
                                        lot_number = match_obj.group(1)
                                else:
                                    lot_number = match_obj.group(0)
                                
                                # Clean up the lot number
                                lot_number = re.sub(r'^(lot|LOT)\s*', '', lot_number, flags=re.IGNORECASE)
                                lot_number = lot_number.strip(' :-\t')
                                
                                # Get the position of this lot number in the full text
                                lot_abs_position = keyword_position - 500 + match_obj.start()
                                
                                if lot_number:
                                    all_lot_matches.append((lot_number, lot_abs_position))
                        
                        # Process unique lots for this keyword match
                        for lot_number, lot_position in all_lot_matches:
                            # Create a unique key for this lot
                            lot_key = f"{lot_number}-{page_num}"
                            
                            # If we haven't seen this lot before, process it
                            if lot_key not in unique_lots:
                                # Extract text starting from the lot number position
                                text_after_lot = text[lot_position:]
                                
                                # Try to extract a meaningful section (title + description)
                                section_text = extract_complete_section(text_after_lot, lot_number)
                                
                                # Store the lot information
                                unique_lots[lot_key] = {
                                    'page': page_num + 1,
                                    'lot_number': lot_number,
                                    'lot_position': lot_position,
                                    'full_text': section_text,
                                    'found_via_keywords': [keyword]  # List of keywords that found this lot
                                }
                            else:
                                # If we've already seen this lot, just add the keyword to the list
                                if keyword not in unique_lots[lot_key]['found_via_keywords']:
                                    unique_lots[lot_key]['found_via_keywords'].append(keyword)
            
            # Convert the dictionary back to a list
            results = list(unique_lots.values())
            return results
            
    except FileNotFoundError:
        print(f"File not found: {pdf_path}")
        return []
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return []

def extract_complete_section(text_after_lot, lot_number):
    """
    Extract a complete section of text after a lot number
    """
    # Look for natural section endings
    end_patterns = [
        r'\n\s*(lot|LOT)\s*[:\-\s]*\s*\d',  # Next lot number
        r'\n\s*\d+\s*\.',  # Next numbered item
        r'\n\s*[A-Z][A-Z\s]+\n',  # ALL CAPS heading
        r'\n\s*[IVX]+\.',  # Roman numerals
    ]
    
    # First, try to find a logical end point
    end_position = len(text_after_lot)
    for pattern in end_patterns:
        match = re.search(pattern, text_after_lot)
        if match and match.start() < end_position:
            end_position = match.start()
    
    # If no natural end found, take a reasonable chunk (first 2000 characters)
    if end_position == len(text_after_lot):
        end_position = min(2000, len(text_after_lot))
    
    section = text_after_lot[:end_position].strip()
    
    # Clean up the text
    section = re.sub(r'\n+', ' ', section)  # Replace multiple newlines with spaces
    section = re.sub(r'\s+', ' ', section)  # Replace multiple spaces with single space
    
    return section

def main():
    pdf_path = r"D:\sorabo\sorabo\pdf\25-124195.pdf"
    
    # Keywords to search for
    keywords = [
        "Serrurerie",
        "45420000",
        "45421141", 
        "44316500",
        "45421000"
    ]
    
    print(f"Searching PDF: {pdf_path}")
    print(f"Keywords: {keywords}")
    print("=" * 80)
    
    results = search_keywords_and_find_lot(pdf_path, keywords)
    
    if results:
        # Group by lot number for better organization (even though they're already unique)
        lot_groups = {}
        for result in results:
            lot_key = f"lot-{result['lot_number']}"
            if lot_key not in lot_groups:
                lot_groups[lot_key] = []
            lot_groups[lot_key].append(result)
        
        # Print results organized by lot number
        for lot_number, occurrences in lot_groups.items():
            print(f"\n{'='*80}")
            print(f"📋 LOT: {lot_number}")
            print(f"{'='*80}")
            
            for i, occurrence in enumerate(occurrences, 1):
                print(f"\n📍 Occurrence {i}:")
                print(f"   Page: {occurrence['page']}")
                print(f"   Found via keywords: {', '.join(occurrence['found_via_keywords'])}")
                print(f"\n   📝 Complete Text:")
                print(f"   {'─'*50}")
                # Print the text with proper formatting
                text_lines = occurrence['full_text'].split('. ')
                for line in text_lines:
                    if line.strip():
                        print(f"   {line.strip()}.")
                print(f"   {'─'*50}")
        
        print(f"\n🎯 Summary: Found {len(results)} unique lot occurrences")
        print(f"📊 Unique lot numbers: {len(lot_groups)}")
        
    else:
        print("\n❌ No results found.")

if __name__ == "__main__":
    main()

Searching PDF: D:\sorabo\sorabo\pdf\25-124195.pdf
Keywords: ['Serrurerie', '45420000', '45421141', '44316500', '45421000']

📋 LOT: lot-0005

📍 Occurrence 1:
   Page: 11
   Found via keywords: 45420000

   📝 Complete Text:
   ──────────────────────────────────────────────────
   LOT-0005 Titre :MENUISERIE EXTERIEURE ALUMINIUM THERMOLAQUE - VITRERIE - PROTECTION SOLAIRE- OCCULTATIONS Description :MENUISERIE EXTERIEURE ALUMINIUM THERMOLAQUE - VITRERIE - PROTECTION SOLAIRE- OCCULTATIONS Identifiant interne :5.
   ──────────────────────────────────────────────────

📋 LOT: lot-0009

📍 Occurrence 1:
   Page: 20
   Found via keywords: 45421000

   📝 Complete Text:
   ──────────────────────────────────────────────────
   LOT-0009 Titre :MENUISERIE INTERIEURE BOIS - MOBILIERS - SIGNALETIQUE Description :MENUISERIE INTERIEURE BOIS - MOBILIERS - SIGNALETIQUE Identifiant interne :9.
   ──────────────────────────────────────────────────

📋 LOT: lot-0010

📍 Occurrence 1:
   Page: 22
   Found via keyw

In [7]:
import PyPDF2
import re

def search_keywords_and_find_lot(pdf_path, keywords):
    """
    Search for keywords in a PDF and find ALL lot numbers that appear before them
    """
    try:
        # Open the PDF file
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            results = []
            
            # Iterate through each page
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                
                # Search for each keyword
                for keyword in keywords:
                    # Find all occurrences of the keyword
                    keyword_matches = list(re.finditer(re.escape(keyword), text, re.IGNORECASE))
                    
                    
                    for match in keyword_matches:
                        keyword_position = match.start()
                        
                        # Extract more text before the keyword (look back up to 500 characters)
                        text_before = text[max(0, keyword_position - 500):keyword_position]
                        
                        # Improved lot pattern to catch more formats
                        lot_patterns = [
                            r'(lot|LOT)\s*[:\-\s]*\s*(\d+[-\w]*)',  # lot: 123, LOT-456, lot 789
                            r'(Lot\s*\d+)',  # Lot 123
                            r'(lot\s*\d+)',  # lot 123
                            r'\b(\d+)\s*-\s*Lot',  # 123 - Lot
                            r'\b(LOT\s*[A-Z]*\d+)',  # LOT A123, LOT 456
                        ]
                        
                        all_lot_matches = []
                        
                        for pattern in lot_patterns:
                            matches = re.findall(pattern, text_before, re.IGNORECASE)
                            for match in matches:
                                if isinstance(match, tuple):
                                    # For patterns that capture groups
                                    lot_number = match[1] if len(match) > 1 else match[0]
                                else:
                                    # For patterns that capture directly
                                    lot_number = match
                                
                                # Clean up the lot number
                                lot_number = re.sub(r'^(lot|LOT)\s*', '', lot_number, flags=re.IGNORECASE)
                                lot_number = lot_number.strip(' :-\t')
                                
                                if lot_number and lot_number not in [lm[0] for lm in all_lot_matches]:
                                    all_lot_matches.append((lot_number, pattern))
                        
                        # Remove duplicates while preserving order
                        unique_lots = []
                        seen = set()
                        for lot_num, pattern in all_lot_matches:
                            if lot_num not in seen:
                                seen.add(lot_num)
                                unique_lots.append(lot_num)
                        
                        if unique_lots:
                            for lot_number in unique_lots:
                                results.append({
                                    'page': page_num + 1,
                                    'keyword': keyword,
                                    'lot_number': lot_number,
                                    'position': keyword_position
                                })
                                print(f"Found: lot-{lot_number} (before '{keyword}' on page {page_num + 1})")
                        else:
                            # Print context for debugging
                            context = text_before[-100:]  # Last 100 chars before keyword
            
            return results
            
    except FileNotFoundError:
        return []
    except Exception as e:
        return []



# Main execution
if __name__ == "__main__":
    pdf_path = r"D:\sorabo\sorabo\pdf\25-124195.pdf"
    
    # Keywords to search for
    keywords = [
        "DEMOLITION"
    ]
    
   
    
    # Method 1: Improved search
    results = search_keywords_and_find_lot(pdf_path, keywords)
    
    # Method 2: Alternative approach
    
    # Final summary
    if results:

        unique_lots = set()
        for result in results:
            unique_lots.add(f"lot-{result['lot_number']}")
        
        for lot in sorted(unique_lots):
            print(lot)
    else:
        print("\nNo results found with Method 1.")
import PyPDF2
import re

def search_keywords_and_find_lot(pdf_path, keywords):
    """
    Search for keywords in a PDF and check if 'visite' appears before them
    """
    try:
        # Open the PDF file
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            
            # Iterate through each page
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                # Search for each keyword
                for keyword in keywords:
                    # Find all occurrences of the keyword
                    keyword_matches = list(re.finditer(re.escape(keyword), text, re.IGNORECASE))
                    
                    for match in keyword_matches:
                        keyword_position = match.start()
                        
                        # Extract text before the keyword (look back up to 500 characters)
                        text_before = text[max(0, keyword_position - 500):keyword_position]
                        
                        # Check if "visite" appears before the keyword
                        visite_patterns = [r"visites", r"visite"]
                        
                        for pattern in visite_patterns:
                            if re.search(pattern, text_before, re.IGNORECASE):
                                return "yes"
            
            return "no"
            
    except FileNotFoundError:
        return "no"
    except Exception as e:
        return "no"

# Main execution
if __name__ == "__main__":
    pdf_path = r"D:\sorabo\sorabo\pdf\25-124195.pdf"
    
    # Keywords to search for
    keywords = ["obligatoires", "obligatoire"]
    
    # Use your method
    result = search_keywords_and_find_lot(pdf_path, keywords)
    
    print(f"Visite obligatoire: {result}")

Found: lot-5 (before 'DEMOLITION' on page 2)
Found: lot-0001 (before 'DEMOLITION' on page 2)
Found: lot-5 (before 'DEMOLITION' on page 2)
Found: lot-0001 (before 'DEMOLITION' on page 2)
lot-0001
lot-5
Visite obligatoire: yes
